In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install git+https://github.com/PetrochukM/PyTorch-NLP.git

  Cloning https://github.com/PetrochukM/PyTorch-NLP.git to /tmp/pip-req-build-llm6k4rp
  Running command git clone --filter=blob:none --quiet https://github.com/PetrochukM/PyTorch-NLP.git /tmp/pip-req-build-llm6k4rp
  Resolved https://github.com/PetrochukM/PyTorch-NLP.git to commit 53d7edcb8e0c099efce7c2ddf8cd7c44157fcac3
  Preparing metadata (setup.py) ... done
  Created wheel for pytorch-nlp: filename=pytorch_nlp-0.5.0-py3-none-any.whl size=88719 sha256=0131054286e1394da92dd5ae1b14fd8f4b57e8eb39978a84005e21f7fe3a56b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-qzsasg8m/wheels/a5/93/b0/9f0138afb1271281613a5af71272c5b246fdd2d421c6fbdf88
Successfully built pytorch-nlp


In [3]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from torchnlp import encoders
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import  mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from pathlib import Path
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


cuda


In [4]:
torch.cuda.empty_cache()

In [5]:
# Load the ROI features (Covid)
train_ROI = torch.load("/kaggle/input/harmeme/MINI_PROJECT_2/harmeme_saved_feat_ROIENT/harmeme_ROI_MOMENTA/cov/memes_harmfulness/harmeme_cov_train_ROI.pt")
val_ROI = torch.load("/kaggle/input/harmeme/MINI_PROJECT_2/harmeme_saved_feat_ROIENT/harmeme_ROI_MOMENTA/cov/memes_harmfulness/harmeme_cov_val_ROI.pt")
test_ROI = torch.load("/kaggle/input/harmeme/MINI_PROJECT_2/harmeme_saved_feat_ROIENT/harmeme_ROI_MOMENTA/cov/memes_harmfulness/harmeme_cov_test_ROI.pt")
# Load the ENT features
train_ENT = torch.load("/kaggle/input/albert-text-features/Train_text_Drob_Feature_albert.pt")
val_ENT = torch.load("/kaggle/input/albert-text-features/Val_text_Drob_Feature_albert.pt")
test_ENT = torch.load("/kaggle/input/albert-text-features/Test_text_Drob_Feature_albert.pt")

In [6]:
data_dir_cov = "/kaggle/input/harmeme/MINI_PROJECT_2/Harm-C/datasets/memes/defaults/images"
train_path_cov = "/kaggle/input/d/yashnikam10/supplementary-data/final_train.json"
dev_path_cov   = "/kaggle/input/d/yashnikam10/supplementary-data/final_val_dict.json"
test_path_cov  = "/kaggle/input/d/yashnikam10/supplementary-data/final_test.json"

In [7]:
train_samples_frame = pd.read_json(train_path_cov, lines=True)
train_samples_frame.head()

,id,image,labels,text,bb_dict
0,covid_memes_18,covid_memes_18.png,"[somewhat harmful, individual]",Bernie or Elizabeth?\nBe informed.Compare them...,"[[0.6680887372013651, 0.632857142857142, 0.825..."
1,covid_memes_19,covid_memes_19.png,"[somewhat harmful, organization]",Extending the\nBrexit deadline until\nOctober ...,[]
2,covid_memes_252,covid_memes_252.png,[not harmful],kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,"[[0.522067189216613, 0.207160174846649, 0.4088..."
3,covid_memes_255,covid_memes_255.png,[not harmful],"So, I order this\nfoce mask to\nprotect ogains...",[]
4,covid_memes_20,covid_memes_20.png,"[somewhat harmful, individual]",best candidate for\nJA\n2020\njoe biden\nKamal...,"[[0.47865853658536506, 0.32835820895522305, 0...."


In [8]:
test_samples_frame = pd.read_json(test_path_cov, lines=True)
test_samples_frame.head()

,id,image,labels,text,bb_dict
0,covid_memes_5425,covid_memes_5425.png,[not harmful],gwen\n@gwenervi\ndis gon be trump tomorrow aft...,[]
1,covid_memes_5426,covid_memes_5426.png,[not harmful],Armani\n@historyofarmani\nBiden after hearing ...,"[[0.071942446043165, 0.2734375, 0.393884892086..."
2,covid_memes_5429,covid_memes_5429.png,[not harmful],MESSAGE FROM TRUMP TO\nCOVID-19\nLEAVE NOW OR ...,"[[0.41454545454545405, 0.125683060109289, 0.64..."
3,covid_memes_5430,covid_memes_5430.png,[not harmful],COVID-19 STARTED DURING HIS TERM\nSOIT SHOULD ...,"[[0.257246376811594, 0.19125683060109203, 0.54..."
4,covid_memes_5434,covid_memes_5434.png,[not harmful],TRUMPS RESPONSE TO COVID-19\nUMP\nTAYM\nINCLUD...,"[[0.25703868269920305, 0.167230486869812, 0.49..."


In [9]:
!pip install ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.3 MB/s eta 0:00:00


In [10]:
import gzip
import html
import os
from functools import lru_cache

import ftfy
import regex as re

In [11]:
!wget -O bpe_simple_vocab_16e6.txt.gz https://drive.google.com/uc?id=1YNkhStUg9YJ3WPUeCWhHyyMUmVqlWrG5

--2023-11-02 15:54:29--  https://drive.google.com/uc?id=1YNkhStUg9YJ3WPUeCWhHyyMUmVqlWrG5
Resolving drive.google.com (drive.google.com)... 74.125.126.101, 74.125.126.139, 74.125.126.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.126.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k4u6j127qmnfhp14iukbiimdnmeatlo8/1698940425000/14204125426768851410/*/1YNkhStUg9YJ3WPUeCWhHyyMUmVqlWrG5?uuid=4cfa1a1d-c97b-4042-b1b4-c39db54a9102 [following]
--2023-11-02 15:54:30--  https://doc-0s-a8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k4u6j127qmnfhp14iukbiimdnmeatlo8/1698940425000/14204125426768851410/*/1YNkhStUg9YJ3WPUeCWhHyyMUmVqlWrG5?uuid=4cfa1a1d-c97b-4042-b1b4-c39db54a9102
Resolving doc-0s-a8-docs.googleusercontent.com (doc-0s-a8-docs.googleusercontent.com)... 173.194.74.132, 2607:f8b0:4001:c0d::84
Connecting to doc-0s

In [12]:
@lru_cache()
def bytes_to_unicode():
    """
    Returns list of utf-8 byte and a corresponding list of unicode strings.
    The reversible bpe codes work on unicode strings.
    This means you need a large # of unicode characters in your vocab if you want to avoid UNKs.
    When you're at something like a 10B token dataset you end up needing around 5K for decent coverage.
    This is a signficant percentage of your normal, say, 32K bpe vocab.
    To avoid that, we want lookup tables between utf-8 bytes and unicode strings.
    And avoids mapping to whitespace/control characters the bpe code barfs on.
    """
    bs = list(range(ord("!"), ord("~")+1))+list(range(ord("¡"), ord("¬")+1))+list(range(ord("®"), ord("ÿ")+1))
    cs = bs[:]
    n = 0
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8+n)
            n += 1
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))


def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def basic_clean(text):
    text = ftfy.fix_text(text)
    text = html.unescape(html.unescape(text))
    return text.strip()


def whitespace_clean(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text


class SimpleTokenizer(object):
    def __init__(self, bpe_path: str = "/kaggle/working/bpe_simple_vocab_16e6.txt.gz"):
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
#         with gzip.open(bpe_path,'w') as t:
#        with gzip.open(bpe_path,'rt', encoding="utf-8") as f:
#                merges = f.read()
        merges = gzip.open(bpe_path).read().decode("utf-8").split('\n')
        #print(merges)
        merges = merges[1:49152-256-2+1]
        merges = [tuple(merge.split()) for merge in merges]
        vocab = list(bytes_to_unicode().values())
        vocab = vocab + [v+'</w>' for v in vocab]
        #print(vocab)
        for merge in merges:
            vocab.append(''.join(merge))
        vocab.extend(['<|startoftext|>', '<|endoftext|>'])
        self.encoder = dict(zip(vocab, range(len(vocab))))
        self.decoder = {v: k for k, v in self.encoder.items()}
        self.bpe_ranks = dict(zip(merges, range(len(merges))))
        self.cache = {'<|startoftext|>': '<|startoftext|>', '<|endoftext|>': '<|endoftext|>'}
        self.pat = re.compile(r"""<\|startoftext\|>|<\|endoftext\|>|'s|'t|'re|'ve|'m|'ll|'d|[\p{L}]+|[\p{N}]|[^\s\p{L}\p{N}]+""", re.IGNORECASE)

    def bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        word = tuple(token[:-1]) + ( token[-1] + '</w>',)
        pairs = get_pairs(word)

        if not pairs:
            return token+'</w>'

        while True:
            bigram = min(pairs, key = lambda pair: self.bpe_ranks.get(pair, float('inf')))
            if bigram not in self.bpe_ranks:
                break
            first, second = bigram
            new_word = []
            i = 0
            while i < len(word):
                try:
                    j = word.index(first, i)
                    new_word.extend(word[i:j])
                    i = j
                except:
                    new_word.extend(word[i:])
                    break

                if word[i] == first and i < len(word)-1 and word[i+1] == second:
                    new_word.append(first+second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            new_word = tuple(new_word)
            word = new_word
            if len(word) == 1:
                break
            else:
                pairs = get_pairs(word)
        word = ' '.join(word)
        self.cache[token] = word
        return word

    def encode(self, text):
        bpe_tokens = []
        text = whitespace_clean(basic_clean(text)).lower()
        for token in re.findall(self.pat, text):
            token = ''.join(self.byte_encoder[b] for b in token.encode('utf-8'))
            bpe_tokens.extend(self.encoder[bpe_token] for bpe_token in self.bpe(token).split(' '))
        return bpe_tokens

    def decode(self, tokens):
        text = ''.join([self.decoder[token] for token in tokens])
        text = bytearray([self.byte_decoder[c] for c in text]).decode('utf-8', errors="replace").replace('</w>', ' ')
        return text

In [13]:
# Downloadig the clip model
MODELS = {
    "RN50": "https://openaipublic.azureedge.net/clip/models/afeb0e10f9e5a86da6080e35cf09123aca3b358a0c3e3b6c78a7b63bc04b6762/RN50.pt",
    "RN101": "https://openaipublic.azureedge.net/clip/models/8fa8567bab74a42d41c5915025a8e4538c3bdbe8804a470a72f30b0d94fab599/RN101.pt",
    "RN50x4": "https://openaipublic.azureedge.net/clip/models/7e526bd135e493cef0776de27d5f42653e6b4c8bf9e0f653bb11773263205fdd/RN50x4.pt",
    "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt",    
}
! wget {MODELS["ViT-B/32"]} -O clip_model.pt

--2023-11-02 15:54:32--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.38, 13.107.213.38, 2620:1ec:bdf::38, ...
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.246.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353976522 (338M) [application/octet-stream]
Saving to: ‘clip_model.pt’

clip_model.pt       100%[===================>] 337.58M   210MB/s    in 1.6s    

2023-11-02 15:54:33 (210 MB/s) - ‘clip_model.pt’ saved [353976522/353976522]



In [14]:
clip_model = torch.jit.load("clip_model.pt").cuda().eval()
input_resolution = clip_model.input_resolution.item()
context_length = clip_model.context_length.item()
vocab_size = clip_model.vocab_size.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in clip_model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [15]:
preprocess = Compose([
    Resize(input_resolution, interpolation=Image.BICUBIC),
    CenterCrop(input_resolution),
    ToTensor()
    ])
tokenizer = SimpleTokenizer()

In [16]:
def process_image_clip(in_img):
    image_mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).cuda()
    image_std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).cuda()
    
    image = preprocess(Image.open(in_img).convert("RGB"))
    
    image_input = torch.tensor(np.stack(image)).cuda()
    image_input -= image_mean[:, None, None]
    image_input /= image_std[:, None, None]
    return image_input

In [17]:
def process_text_clip(in_text):    
    text_token = tokenizer.encode(in_text)
    text_input = torch.zeros(clip_model.context_length, dtype=torch.long)
    sot_token = tokenizer.encoder['<|startoftext|>']
    eot_token = tokenizer.encoder['<|endoftext|>']
    tokens = [sot_token] + text_token[:75] + [eot_token]
    text_input[:len(tokens)] = torch.tensor(tokens)
    text_input = text_input.cuda()
    return text_input

In [18]:
import torch
from torch import optim, nn
from torchvision import models, transforms
import cv2

# YOLOv8

In [19]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 10.8 MB/s eta 0:00:00a 0:00:01


In [20]:
from ultralytics.nn.tasks import DetectionModel
class YOLOv8DetectionAndFeatureExtractorModel(DetectionModel):
    def __init__(self, cfg='yolov8n.yaml', ch=3, nc=None, verbose=True):  # model, input channels, number of classes
        super().__init__(cfg, ch, nc, verbose)
    
    def custom_forward(self, x):
        """
        This is a modified version of the original _forward_once() method in BaseModel,
        found in ultralytics/nn/tasks.py.
        The original method returns only the detection output, while this method returns
        both the detection output and the features extracted by the last convolutional layer.
        """
        y = []
        features = None
        for m in self.model:
            if m.f != -1:  # if not from previous layer
                x = y[m.f] if isinstance(m.f, int) else [x if j == -1 else y[j] for j in m.f]  # from earlier layers
            if torch.is_tensor(x):
                features = x # keep the last tensor as features
            x = m(x)  # run
            if torch.is_tensor(x):
                features = x # keep the last tensor as features
            y.append(x if m.i in self.save else None)  # save output
        if torch.is_tensor(x):
            features = x # keep the last tensor as features
        return features, x # return features and detection output

def create_yolov8_model(model_name_or_path,nc):
    from ultralytics.nn.tasks import attempt_load_one_weight
    #from ultralytics.yolo.cfg import get_cfg
    ckpt = None
    if str(model_name_or_path).endswith('.pt'):
        weights, ckpt = attempt_load_one_weight(model_name_or_path)
        cfg = ckpt['model'].yaml
    else:
        cfg = model_name_or_path
    model = YOLOv8DetectionAndFeatureExtractorModel(cfg, nc=nc, verbose=True)
    return model

In [21]:
model_yolov8 = create_yolov8_model('yolov8n.pt', 80)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
model_yolov8 = model_yolov8.to(device)

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.1MB/s]

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    2954

In [22]:
from tqdm import tqdm
import numpy as np

In [23]:
transform_yolov8_BB = transforms.Compose([
  transforms.ToPILImage(),
#   transforms.CenterCrop(512),
  transforms.Resize((640,640)),
  transforms.ToTensor()                              
])

In [24]:
from tqdm import tqdm
import numpy as np

# Transform the image, so it becomes readable with the model
transform_yolov8_center = transforms.Compose([
  transforms.ToPILImage(),
  transforms.CenterCrop(640),
  transforms.Resize(640),
  transforms.ToTensor()                              
])

In [25]:
def get_image_yolov8_BB(l, t, r, b, in_im): 
#     left, top, right, bottom and input image
    img = cv2.imread(in_im)
    h, w, _ = img.shape
    # crop
    x1 = int(np.floor(l*w))
    x2 = int(np.floor(r*w))
    y1 = int(np.floor(b*h))
    y2 = int(np.floor(t*h))
    crop_img = img[y1:y2, x1:x2]    
    
    # Transform the cropped image
    img = transform_yolov8_BB(crop_img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 640, 640)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
        # Extract the feature from the image
        feature = model_yolov8.custom_forward(img)[0].flatten()
    # Convert to NumPy Array, Reshape it, and save it to features variable
    return feature

In [26]:
def get_image_yolov8_center(in_im):
    # Set the image path
    path = in_im
    # Read the file
    img = cv2.imread(path)
    # Transform the image
    img = transform_yolov8_center(img)
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 640, 640)
    img = img.to(device)
    # We only extract features, so we don't need gradient
    with torch.no_grad():
        # Extract the feature from the image
        feature = model_yolov8.custom_forward(img)[0].flatten()
    
    return feature

In [27]:
# !pip install git+https://github.com/UKPLab/sentence-transformers

In [28]:
# from sentence_transformers import SentenceTransformer
# model_sent_trans = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [29]:
class HarmemeMemesDatasetAug2(torch.utils.data.Dataset):
    """Uses jsonl data to preprocess and serve 
    dictionary of multimodal tensors for model input.
    """

    def __init__(
        self,
        data_path,
        img_dir,
        split_flag=None,
        balance=False,
        dev_limit=None,
        random_state=0,
    ):

        self.samples_frame = pd.read_json(
            data_path, lines=True
        )
        self.samples_frame = self.samples_frame.reset_index(
            drop=True
        )
        self.samples_frame.image = self.samples_frame.apply(
            lambda row: (img_dir + '/' + row.image), axis=1
        )
        if split_flag=='train':
            self.ROI_samples = train_ROI
            self.ENT_samples = train_ENT
        elif split_flag=='val':
            self.ROI_samples = val_ROI
            self.ENT_samples = val_ENT
        else:
            self.ROI_samples = test_ROI
            self.ENT_samples = test_ENT
        
    def __len__(self):
        """This method is called when you do len(instance) 
        for an instance of this class.
        """
        return len(self.samples_frame)

    def __getitem__(self, idx):
        """This method is called when you do instance[key] 
        for an instance of this class.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_id = self.samples_frame.loc[idx, "id"]
        img_file_name = self.samples_frame.loc[idx, "image"]
        
        image_clip_input = process_image_clip(self.samples_frame.loc[idx, "image"])
# --------------------------------------------------------------------------------------        
#         Pre-extracted features
#         image_vgg_feature = self.ROI_samples[idx]        
# --------------------------------------------------------------------------------------
# On-demand computation
        BB_info = self.samples_frame.loc[idx, "bb_dict"]
        roi_yolov8_feat_list = []
        if BB_info:
            total_BB = len(BB_info)
            if total_BB>8:
                BB_info_final = BB_info[:8]
            else:
                BB_info_final = BB_info
#             Have to get VGG reps for each cropped BB and get the mean             
            for item in BB_info_final:
#                 Get the top left (left,top) and bottom right (right,bottom) values of the coordinates
#                 top left and bottom right value extraction                
                left   = item[0]
                bottom = item[1]
                top    = bottom + item[3]
                right  = left + item[2]
                #get_image_yolov8_center(img_file_name)   # ??
                roi_yolov8_feat = get_image_yolov8_BB(left, top, right, bottom, img_file_name)
                #print(roi_vgg_feat.shape)
                roi_yolov8_feat_list.append(roi_yolov8_feat)
#             print(np.shape(roi_vgg_feat_list))
#             print(torch.cat(roi_vgg_feat_list, dim=0))
#             print(np.mean(np.array(roi_vgg_feat_list), axis=0))
            image_yolov8_feature = torch.mean(torch.vstack(roi_yolov8_feat_list), axis=0)
#             print(image_vgg_feature.shape)
        else:
            image_yolov8_feature = torch.tensor(get_image_yolov8_center(img_file_name))
# --------------------------------------------------------------------------------------
        text_clip_input = process_text_clip(self.samples_frame.loc[idx, "text"])
#         -------------------------------------------------------------------------------
#         Process entities
        #         Use them directly from the saved files
        text_drob_feature = self.ENT_samples[idx]
#         -------------------------------------------------------------------------------
# #         Get the mean representation for the set of entities ""on-demand
#         cur_ent_rep_list = []
#         cur_ent_list = self.samples_frame.loc[idx, "ent"]
        
#         if len(cur_ent_list):
#             for item in cur_ent_list:
#                 cur_ent_rep = torch.tensor(model_sent_trans.encode(item)).to(device)
#                 cur_ent_rep_list.append(cur_ent_rep)
#             text_drob_feature = torch.mean(torch.vstack(cur_ent_rep_list), axis=0)
#         else:
#             text_drob_feature = torch.tensor(model_sent_trans.encode(self.samples_frame.loc[idx, "text"])).to(device)
#         -------------------------------------------------------------------------------

        if "labels" in self.samples_frame.columns:
#             label = torch.Tensor(
#                 [self.samples_frame.loc[idx, "label"]]
#             ).long().squeeze()

#             Uncoment below for binary index creation
#             if self.samples_frame.loc[idx, "labels"][0]=="not harmful":
#                 lab=0
#             else:
#                 lab=1            
#             label = torch.tensor(lab).to(device)  

#             Uncomment below for one hot encoding
#             y = torch.tensor(lab).to(device)
#             label = F.one_hot(y, num_classes=2)  

# #             Multiclass setting - harmfulness
            if self.samples_frame.loc[idx, "labels"][0]=="not harmful":
                lab=0
            elif self.samples_frame.loc[idx, "labels"][0]=="somewhat harmful":
                lab=1  
            else:
                lab=2
            label = torch.tensor(lab).to(device)  

            
            sample = {
                "id": img_id, 
                "image_clip_input": image_clip_input,
                "image_yolov8_feature": image_yolov8_feature,
                "text_clip_input": text_clip_input,
                "text_drob_embedding": text_drob_feature,
                "label": label
            }
        else:
            sample = {
                "id": img_id, 
                "image_clip_input": image_clip_input,
                "image_yolov8_feature": image_yolov8_feature,
                "text_clip_input": text_clip_input,
                "text_drob_embedding": text_drob_feature
            }

        return sample

In [30]:
hm_dataset_train = HarmemeMemesDatasetAug2(train_path_cov, data_dir_cov, 'train')
dataloader_train = DataLoader(hm_dataset_train, batch_size=64,
                        shuffle=True, num_workers=0)
hm_dataset_val = HarmemeMemesDatasetAug2(dev_path_cov, data_dir_cov, 'val')
dataloader_val = DataLoader(hm_dataset_val, batch_size=64,
                        shuffle=True, num_workers=0)
hm_dataset_test = HarmemeMemesDatasetAug2(test_path_cov, data_dir_cov, 'test')
dataloader_test = DataLoader(hm_dataset_test, batch_size=64,
                        shuffle=False, num_workers=0)

In [31]:
class MM(nn.Module):
    def __init__(self, n_out):
        super(MM, self).__init__()  
        
        self.dense_yolov8_4096 = nn.Linear(102400, 4096)
        self.dense_vgg_1024 = nn.Linear(4096, 1024)
        self.dense_vgg_512 = nn.Linear(1024, 512)
        self.drop50 = nn.Dropout(p=0.5)
        self.drop20 = nn.Dropout(p=0.2)
        self.drop5 = nn.Dropout(p=0.05) 
        
        self.dense_drob_512 = nn.Linear(768, 512)
        
        self.gen_key_L1 = nn.Linear(512, 256) # 512X256
        self.gen_query_L1 = nn.Linear(512, 256) # 512X256
        self.gen_key_L2 = nn.Linear(512, 256) # 512X256
        self.gen_query_L2 = nn.Linear(512, 256) # 512X256
        self.gen_key_L3 = nn.Linear(512, 256) # 512X256
        self.gen_query_L3 = nn.Linear(512, 256) # 512X256
#         self.gen_value = nn.Linear(512, 256) # 512X256
        self.soft = nn.Softmax(dim=1)
        self.soft_final = nn.Softmax(dim=1)
        self.project_dense_512a = nn.Linear(1024, 512) # 512X256
        self.project_dense_512b = nn.Linear(1024, 512) # 512X256
        self.project_dense_512c = nn.Linear(1024, 512) # 512X256 
        
        
        self.fc_out = nn.Linear(512, 256) # 512X256
        self.out = nn.Linear(256, n_out) # 512X256
        

    def selfattNFuse_L1a(self, vec1, vec2): 
            q1 = F.relu(self.gen_query_L1(vec1))
            k1 = F.relu(self.gen_key_L1(vec1))
            q2 = F.relu(self.gen_query_L1(vec2))
            k2 = F.relu(self.gen_key_L1(vec2))
            score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
            score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
            wt_score1_score2_mat = torch.cat((score1, score2), 1)
            wt_i1_i2 = self.soft(wt_score1_score2_mat.float()) #prob
            prob_1 = wt_i1_i2[:,0]
            prob_2 = wt_i1_i2[:,1]
            wtd_i1 = vec1 * prob_1[:, None]
            wtd_i2 = vec2 * prob_2[:, None]
            out_rep = F.relu(self.project_dense_512a(torch.cat((wtd_i1,wtd_i2), 1)))
            return out_rep
    def selfattNFuse_L1b(self, vec1, vec2): 
            q1 = F.relu(self.gen_query_L2(vec1))
            k1 = F.relu(self.gen_key_L2(vec1))
            q2 = F.relu(self.gen_query_L2(vec2))
            k2 = F.relu(self.gen_key_L2(vec2))
            score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
            score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
            wt_score1_score2_mat = torch.cat((score1, score2), 1)
            wt_i1_i2 = self.soft(wt_score1_score2_mat.float()) #prob
            prob_1 = wt_i1_i2[:,0]
            prob_2 = wt_i1_i2[:,1]
            wtd_i1 = vec1 * prob_1[:, None]
            wtd_i2 = vec2 * prob_2[:, None]
            out_rep = F.relu(self.project_dense_512b(torch.cat((wtd_i1,wtd_i2), 1)))
            return out_rep
    
    def selfattNFuse_L2(self, vec1, vec2): 
            q1 = F.relu(self.gen_query_L3(vec1))
            k1 = F.relu(self.gen_key_L3(vec1))
            q2 = F.relu(self.gen_query_L3(vec2))
            k2 = F.relu(self.gen_key_L3(vec2))
            score1 = torch.reshape(torch.bmm(q1.view(-1, 1, 256), k2.view(-1, 256, 1)), (-1, 1))
            score2 = torch.reshape(torch.bmm(q2.view(-1, 1, 256), k1.view(-1, 256, 1)), (-1, 1))
            wt_score1_score2_mat = torch.cat((score1, score2), 1)
            wt_i1_i2 = self.soft(wt_score1_score2_mat.float()) #prob
            prob_1 = wt_i1_i2[:,0]
            prob_2 = wt_i1_i2[:,1]
            wtd_i1 = vec1 * prob_1[:, None]
            wtd_i2 = vec2 * prob_2[:, None]
            out_rep = F.relu(self.project_dense_512c(torch.cat((wtd_i1,wtd_i2), 1)))
            return out_rep


    def forward(self, in_CI, in_VGG, in_CT, in_Drob):        
        YOLOV8_feat = self.drop20(F.relu(self.dense_vgg_512(self.drop20(F.relu(self.dense_vgg_1024(self.drop50(F.relu(self.dense_yolov8_4096(in_VGG)))))))))
        Drob_feat = self.drop5(F.relu(self.dense_drob_512(in_Drob)))
        out_img = self.selfattNFuse_L1a(YOLOV8_feat, in_CI)
        out_txt = self.selfattNFuse_L1b(Drob_feat, in_CT)        
        out_img_txt = self.selfattNFuse_L2(out_img, out_txt)
        final_out = F.relu(self.fc_out(out_img_txt))
#         out = torch.sigmoid(self.out(final_out)) #For binary case
        out = self.out(final_out)
        return out

In [32]:
# output_size = 1 #Binary case
output_size = 3
exp_name = "EMNLP_MCHarm_GLAREAll_COVTrain_POLEval"
# pre_trn_ckp = "EMNLP_MCHarm_GLAREAll_COVTrain" # Uncomment for using pre-trained
exp_path = "/kaggle/working/EMNLP_ModelCkpt/"+exp_name
lr=0.001
# criterion = nn.BCELoss() #Binary case
criterion = nn.CrossEntropyLoss()
# # ------------Fresh training------------
model = MM(output_size)
model.to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)

MM(
  (dense_yolov8_4096): Linear(in_features=102400, out_features=4096, bias=True)
  (dense_vgg_1024): Linear(in_features=4096, out_features=1024, bias=True)
  (dense_vgg_512): Linear(in_features=1024, out_features=512, bias=True)
  (drop50): Dropout(p=0.5, inplace=False)
  (drop20): Dropout(p=0.2, inplace=False)
  (drop5): Dropout(p=0.05, inplace=False)
  (dense_drob_512): Linear(in_features=768, out_features=512, bias=True)
  (gen_key_L1): Linear(in_features=512, out_features=256, bias=True)
  (gen_query_L1): Linear(in_features=512, out_features=256, bias=True)
  (gen_key_L2): Linear(in_features=512, out_features=256, bias=True)
  (gen_query_L2): Linear(in_features=512, out_features=256, bias=True)
  (gen_key_L3): Linear(in_features=512, out_features=256, bias=True)
  (gen_query_L3): Linear(in_features=512, out_features=256, bias=True)
  (soft): Softmax(dim=1)
  (soft_final): Softmax(dim=1)
  (project_dense_512a): Linear(in_features=1024, out_features=512, bias=True)
  (project_dens

In [33]:
! nvcc --version
! nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Thu Nov  2 15:54:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W /  70W |   2596MiB / 15109MiB |      0%      Default |
|                               |       

In [34]:
!git clone https://github.com/Bjarten/early-stopping-pytorch

Cloning into 'early-stopping-pytorch'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Receiving objects: 100% (92/92), 533.89 KiB | 3.79 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [35]:
!pip install -r early-stopping-pytorch/requirements.txt

In [36]:
!cp early-stopping-pytorch/pytorchtools.py .

In [37]:
# import EarlyStopping
import os, sys
#sys.path.append('/opt/conda/lib/python3.10/site-packages/pytorchtools/')
from pytorchtools import EarlyStopping

In [38]:
def train_model(model, patience, n_epochs):
    epochs = n_epochs
#     clip = 5

    train_acc_list=[]
    val_acc_list=[]
    train_loss_list=[]
    val_loss_list=[]
    
        # initialize the experiment path
    Path(exp_path).mkdir(parents=True, exist_ok=True)
    # initialize early_stopping object
    chk_file = os.path.join(exp_path, 'checkpoint_'+exp_name+'.pt')
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=chk_file)


    model.train()
    for i in range(epochs):
#         total_acc_train = 0
        total_loss_train = 0
        total_train = 0
        correct_train = 0

        for data in dataloader_train:
            
#             Clip features...
            img_inp_clip = data['image_clip_input']
            txt_inp_clip = data['text_clip_input']
            with torch.no_grad():
                img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)

            img_feat_yolov8 = data['image_yolov8_feature']
            txt_feat_trans = data['text_drob_embedding']

            label_train = data['label'].to(device)

            model.zero_grad()
            
            output = model(img_feat_clip, img_feat_yolov8, txt_feat_clip, txt_feat_trans)
#             print(output.shape)
#             output = model(img_feat_vgg, txt_feat_trans)

            loss = criterion(output.squeeze(), label_train)
            
#             print(loss)
            loss.backward()
#             nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            with torch.no_grad():
                _, predicted_train = torch.max(output.data, 1)
                total_train += label_train.size(0)
                correct_train += (predicted_train == label_train).sum().item()
#                 out_val = (output.squeeze()>0.5).float()
#                 out_final = ((out_val == 1).nonzero(as_tuple=True)[0])
#                 print()
#                 acc = torch.abs(output.squeeze() - label.float()).view(-1)
#                 acc = (1. - acc.sum() / acc.size()[0])
#                 total_acc_train += acc
                total_loss_train += loss.item()

        
        train_acc = 100 * correct_train / total_train
        train_loss = total_loss_train/total_train
        model.eval()
#         total_acc_val = 0
        total_loss_val = 0
        total_val = 0
        correct_val = 0

        with torch.no_grad():
            for data in dataloader_val:                
#                 Clip features...                
                img_inp_clip = data['image_clip_input']
                txt_inp_clip = data['text_clip_input']
                with torch.no_grad():
                    img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                    txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)
                
                
                img_feat_yolov8 = data['image_yolov8_feature']                
                txt_feat_trans = data['text_drob_embedding']

                

                label_val = data['label'].to(device)

                model.zero_grad()
                
                output = model(img_feat_clip, img_feat_yolov8, txt_feat_clip, txt_feat_trans)
#                 output = model(img_feat_vgg, txt_feat_trans)
                
                
                val_loss = criterion(output.squeeze(), label_val)
                _, predicted_val = torch.max(output.data, 1)
                total_val += label_val.size(0)
                correct_val += (predicted_val == label_val).sum().item()                
                total_loss_val += val_loss.item()
        print("Saving model...") 
        torch.save(model.state_dict(), os.path.join(exp_path, "final.pt"))

        val_acc = 100 * correct_val / total_val
        val_loss = total_loss_val/total_val

        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        
        early_stopping(val_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
            
        print(f'Epoch {i+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
        model.train()
        torch.cuda.empty_cache()
        
    # load the last checkpoint with the best model
#     model.load_state_dict(torch.load('checkpoint_1.pt'))
    
    return  model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, i

In [39]:
def test_model(model):
    model.eval()
    total_test = 0
    correct_test =0
    total_acc_test = 0
    total_loss_test = 0
    outputs = []
    test_labels=[]
    with torch.no_grad():
        for data in dataloader_test:
            img_inp_clip = data['image_clip_input']
            txt_inp_clip = data['text_clip_input']
            with torch.no_grad():
                img_feat_clip = clip_model.encode_image(img_inp_clip).float().to(device)
                txt_feat_clip = clip_model.encode_text(txt_inp_clip).float().to(device)

            img_feat_yolov8 = data['image_yolov8_feature']
            txt_feat_trans = data['text_drob_embedding']            

            label_test = data['label'].to(device)
            
#             out = model(img_feat_vgg, txt_feat_trans)        

            out = model(img_feat_clip, img_feat_yolov8, txt_feat_clip, txt_feat_trans)        

            outputs += list(out.cpu().data.numpy())
            loss = criterion(out.squeeze(), label_test)
            
            _, predicted_test = torch.max(out.data, 1)
            total_test += label_test.size(0)
            correct_test += (predicted_test == label_test).sum().item()
#                 out_val = (output.squeeze()>0.5).float()
#                 out_final = ((out_val == 1).nonzero(as_tuple=True)[0])
#                 print()
#                 acc = torch.abs(output.squeeze() - label.float()).view(-1)
#                 acc = (1. - acc.sum() / acc.size()[0])
#                 total_acc_train += acc
            total_loss_test += loss.item()
            
            
#     #         print(label.float())
#             acc = torch.abs(out.squeeze() - label.float()).view(-1)
#     #         print((acc.sum() / acc.size()[0]))
#             acc = (1. - acc.sum() / acc.size()[0])
#     #         print(acc)
#             total_acc_test += acc
#             total_loss_test += loss.item()

    
    acc_test = 100 * correct_test / total_test
    loss_test = total_loss_test/total_test   
    
    print(f'acc: {acc_test:.4f} loss: {loss_test:.4f}')
    return outputs

In [40]:
import warnings
warnings.filterwarnings("ignore")

In [41]:
epoc_num_total = 0
train_acc_list_full = []
val_acc_list_full = []
train_loss_list_full = []
val_loss_list_full = []

In [42]:
n_epochs = 3
# early stopping patience; how long to wait after last time validation loss improved.
patience = 1
model, train_acc_list, val_acc_list, train_loss_list, val_loss_list, epoc_num = train_model(model, patience, n_epochs)
epoc_num_total += epoc_num
train_acc_list_full += train_acc_list
val_acc_list_full += val_acc_list
train_loss_list_full += train_loss_list
val_loss_list_full += val_loss_list

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Saving model...
Validation loss decreased (inf --> 0.011006).  Saving model ...
Epoch 1: train_loss: 0.0113 train_acc: 69.7975 | val_loss: 0.0110 val_acc: 75.7062


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known inc

Saving model...
EarlyStopping counter: 1 out of 1
Early stopping


In [43]:
# import matplotlib.pyplot as plt
# get_ipython().run_line_magic('matplotlib', 'inline')
# epochs = range(epoc_num_total+1)
# train_acc_list
# val_acc_list
# train_loss_list
# val_loss_list
# # plt.plot(epochs, train_acc_list)
# # plt.plot(epochs, val_acc_list)
# fig1, ax1 = plt.subplots()
# ax1.plot(epochs, train_acc_list_full, label="train acc")
# ax1.plot(epochs, val_acc_list_full, label="val acc")
# ax1.set_title("accuracy plot")
# ax1.set_xlabel("epochs")
# ax1.legend(loc="upper left")
# fig2, ax2 = plt.subplots()
# ax2.plot(epochs, train_loss_list_full, label="train loss")
# ax2.plot(epochs, val_loss_list_full, label="val loss")
# ax2.set_title("loss plot")
# ax2.set_xlabel("epochs")
# ax2.legend(loc="upper left")

In [44]:
outputs = test_model(model)

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


acc: 79.6610 loss: 0.0094


libpng warning: iCCP: known incorrect sRGB profile


In [45]:
y_pred=[]
for i in outputs:
#     print(np.argmax(i))
    y_pred.append(np.argmax(i))
# # np.argmax(outputs[:])
# outputs

# # Multiclass setting
test_labels=[]
for index, row in test_samples_frame.iterrows():
    lab = row['labels'][0]
    if lab=="not harmful":
        test_labels.append(0)
    elif lab=="somewhat harmful":
        test_labels.append(1)
    else:
        test_labels.append(2)

In [46]:
def calculate_mmae(expected, predicted, classes):
    NUM_CLASSES = len(classes)
    count_dict = {}
    dist_dict = {}
    for i in range(NUM_CLASSES):
        count_dict[i] = 0
        dist_dict[i] = 0.0
    for i in range(len(expected)):
        dist_dict[expected[i]] += abs(expected[i] - predicted[i])
        count_dict[expected[i]] += 1
    overall = 0.0
    for claz in range(NUM_CLASSES): 
        class_dist =  1.0 * dist_dict[claz] / count_dict[claz] 
        overall += class_dist
    overall /= NUM_CLASSES
#     return overall[0]
    return overall

In [47]:
rec = np.round(recall_score(test_labels, y_pred, average="macro"),4)
prec = np.round(precision_score(test_labels, y_pred, average="macro"),4)
f1 = np.round(f1_score(test_labels, y_pred, average="macro"),4)
# hl = np.round(hamming_loss(test_labels, y_pred),4)
acc = np.round(accuracy_score(test_labels, y_pred),4)
mmae = np.round(calculate_mmae(test_labels, y_pred, [0,1,2]),4)
mae = np.round(mean_absolute_error(test_labels, y_pred),4)
# print("recall_score\t: ",rec)
# print("precision_score\t: ",prec)
# print("f1_score\t: ",f1)
# print("hamming_loss\t: ",hl)
# print("accuracy_score\t: ",f1)
print(classification_report(test_labels, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       230
           1       0.62      0.83      0.71       103
           2       0.00      0.00      0.00        21

    accuracy                           0.80       354
   macro avg       0.51      0.56      0.53       354
weighted avg       0.77      0.80      0.78       354



In [48]:
print("Acc, F1, Rec, Prec, MAE, MMAE")
print(acc, f1, rec, prec, mae, mmae)

Acc, F1, Rec, Prec, MAE, MMAE
0.7966 0.5303 0.5606 0.5096 0.2119 0.487
